In [1]:
import keras
from keras import layers

import tensorflow as tf
from tensorflow.python.ops import math_ops 

import math
import numpy as np
import pandas as pd
from tensorflow import data as tf_data
import matplotlib.pyplot as plt
from functools import partial

In [4]:
df = pd.read_csv('../dataset/datasets_for_training/auto_label_data_augmented_2.csv')

print(df.head())

df2 = pd.read_csv('../dataset/datasets_for_pandas/tt_dataset_agg1.csv')
print(df2.head())

                                         sensor_data   label
0  $$$0.9480900232691144/0.6571609099218534/0.516...  Normal
1  $$$0.9532251714291355/0.6622960580818745/0.521...  Normal
2  $$$1.0/0.4956037531490986/0.6086395773571097/0...  Normal
3  $$$1.0002667230831903/0.49587047623228897/0.60...  Normal
4  $$$0.9391821067266011/0.9524225792190245/0.342...  Normal
   accelerometerXAxis  accelerometerYAxis  accelerometerZAxis  gyroscopeXAxis  \
0            0.948090            0.657161            0.516222        0.343511   
1            1.000000            0.495604            0.608640        0.312977   
2            0.939182            0.952423            0.342116        0.503817   
3            0.548321            0.729706            0.591897        0.748092   
4            0.945194            0.538609            0.751990        0.610687   

   gyroscopeYAxis  gyroscopeZAxis       label  
0        0.545455        1.000000  Aggressive  
1        0.378788        0.880000  Aggressive  
2  